In [57]:
library(tidyverse)
library(reshape2)
library(data.table)
library(dplyr)
library(knitr)

In [58]:
library("RSQLite")

## connect to db
con <- dbConnect(drv=RSQLite::SQLite(), dbname="../input/soccer/database.sqlite")

## list all tables
tables <- dbListTables(con)

## exclude sqlite_sequence (contains table information)
tables <- tables[tables != "sqlite_sequence"]
tables
lDataFrames <- vector("list", length=length(tables))

## create a data.frame for each table
for (i in seq(along=tables)) {
  lDataFrames[[i]] <- dbGetQuery(conn=con, statement=paste("SELECT * FROM '", tables[[i]], "'", sep=""))
}

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"

In [59]:
# create dataframes
country <-  data.frame(lDataFrames[1])
league  <-  data.frame(lDataFrames[2])
match   <-  data.frame(lDataFrames[3])
player  <-  data.frame(lDataFrames[4])
team    <-  data.frame(lDataFrames[6])

# select relevant columns
country <- select(country, id, name) %>% rename(country_id = id)  %>% rename(country_name = name)   # use country_id as key for join
league  <- select(league, country_id, name) %>% rename(league_name = name) # use country_id as key for join
match   <- select(match, id, country_id, league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11, goal, shoton, shotoff, foulcommit, card, cross, corner, possession)
player  <- select(player,player_api_id, player_name) # use player_api_id as key for join
team    <- select(team, team_api_id, team_long_name, team_short_name) # use team_api_id as key for join


In [60]:
others = colnames(match)[1:11]
players = colnames(match)[12:33]

In [61]:
# flatten the data
flatten <- melt(match, id = others, measure = players, na.rm = TRUE, value.name = "player_api_id") %>% 
  mutate(team_api_id = 
         ifelse(grepl("home",variable),home_team_api_id, 
         ifelse(grepl("away",variable),away_team_api_id,NA))) %>%  
  left_join(country, by = "country_id")    %>% 
  left_join(league,  by = "country_id")    %>%  
  left_join(team,    by = "team_api_id")   %>%
  left_join(player,  by = "player_api_id") %>% 
  separate(season, into = c("season_start","season_end"), sep = "/", convert = TRUE) 
head(flatten)

Warning message in melt(match, id = others, measure = players, na.rm = TRUE, value.name = "player_api_id"):
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(match). In the next version, this warning will become an error.”


,id,country_id,league_id,season_start,season_end,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,variable,player_api_id,team_api_id,country_name,league_name,team_long_name,team_short_name,player_name
,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,145,1,1,2008,2009,24,2009-02-28 00:00:00,493016,9996,8635,1,1,home_player_1,39890,9996,Belgium,Belgium Jupiler League,Royal Excel Mouscron,MOU,Mark Volders
2,146,1,1,2008,2009,24,2009-02-27 00:00:00,493017,8203,9987,2,1,home_player_1,38327,8203,Belgium,Belgium Jupiler League,KV Mechelen,MEC,Wouter Biebauw
3,147,1,1,2008,2009,24,2009-02-28 00:00:00,493018,9986,9998,3,0,home_player_1,95597,9986,Belgium,Belgium Jupiler League,Sporting Charleroi,CHA,Bertrand Laquait
4,149,1,1,2008,2009,24,2009-03-01 00:00:00,493020,9994,9991,0,1,home_player_1,30934,9994,Belgium,Belgium Jupiler League,Sporting Lokeren,LOK,Boubacar Barry Copa
5,150,1,1,2008,2009,24,2009-02-28 00:00:00,493021,8342,9999,2,1,home_player_1,37990,8342,Belgium,Belgium Jupiler League,Club Brugge KV,CLB,Stijn Stijnen
6,151,1,1,2008,2009,24,2009-02-28 00:00:00,493022,9993,8571,3,0,home_player_1,38391,9993,Belgium,Belgium Jupiler League,Beerschot AC,BAC,Silvio Proto


In [62]:
team_players <- select(flatten, season_start, season_end, country_name, league_name, player_api_id, team_api_id, team_long_name, team_short_name, player_name)
head(team_players)

,season_start,season_end,player_api_id,team_api_id,team_long_name,team_short_name,player_name,country_name,league_name
,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2008,2009,39890,9996,Royal Excel Mouscron,MOU,Mark Volders,Belgium,Belgium Jupiler League
2,2008,2009,38327,8203,KV Mechelen,MEC,Wouter Biebauw,Belgium,Belgium Jupiler League
3,2008,2009,95597,9986,Sporting Charleroi,CHA,Bertrand Laquait,Belgium,Belgium Jupiler League
4,2008,2009,30934,9994,Sporting Lokeren,LOK,Boubacar Barry Copa,Belgium,Belgium Jupiler League
5,2008,2009,37990,8342,Club Brugge KV,CLB,Stijn Stijnen,Belgium,Belgium Jupiler League
6,2008,2009,38391,9993,Beerschot AC,BAC,Silvio Proto,Belgium,Belgium Jupiler League


In [75]:
team_players_2015 <- team_players %>% filter(season_start == 2015)
head(team_players_2015)
team_players_2014 <- team_players %>% filter(season_start == 2014)
head(team_players_2014)

,season_start,season_end,player_api_id,team_api_id,team_long_name,team_short_name,player_name,country_name,league_name
,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2015,2016,242243,9997,Sint-Truidense VV,STT,William Dutoit,Belgium,Belgium Jupiler League
2,2015,2016,206592,8571,KV Kortrijk,KOR,Darren Keet,Belgium,Belgium Jupiler League
3,2015,2016,91929,9987,KRC Genk,GEN,Laszlo Koteles,Belgium,Belgium Jupiler League
4,2015,2016,107806,8573,KV Oostende,OOS,Didier Ovono,Belgium,Belgium Jupiler League
5,2015,2016,37900,10000,SV Zulte-Waregem,ZUL,Sammy Bossuyt,Belgium,Belgium Jupiler League
6,2015,2016,346114,10001,KVC Westerlo,WES,Koen Van Langendonck,Belgium,Belgium Jupiler League


,season_start,season_end,player_api_id,team_api_id,team_long_name,team_short_name,player_name,country_name,league_name
,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2014,2015,20747,9985,Standard de Liège,STL,Eiji Kawashima,Belgium,Belgium Jupiler League
2,2014,2015,208699,8475,Waasland-Beveren,WAA,Colin Coosemans,Belgium,Belgium Jupiler League
3,2014,2015,67949,9984,KSV Cercle Brugge,CEB,Olivier Werner,Belgium,Belgium Jupiler League
4,2014,2015,270473,9989,Lierse SK,LIE,Igor Berezovsky,Belgium,Belgium Jupiler League
5,2014,2015,346114,10001,KVC Westerlo,WES,Koen Van Langendonck,Belgium,Belgium Jupiler League
6,2014,2015,13131,8203,KV Mechelen,MEC,Tomislav Pacovski,Belgium,Belgium Jupiler League
